# HW02 - Sentiment Analysis

In [67]:
from bs4 import BeautifulSoup
import nltk
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
import pandas as pd

In [83]:
categories = ['books', 'dvd', 'electronics', 'kitchen']
files = ['positive', 'negative', 'unlabeled']

In [84]:
def create_labeled_dataset(category):
    dfs = []
    for file in files:
        with open('./data/SA/' + category + '/' + file + '.review', encoding = 'ISO-8859-1') as fp:
            soup = BeautifulSoup(fp, "html.parser")
        reviews = soup.find_all("review_text")
        text = []
        tag = []
        for review in reviews:
            text.append(review.text.strip('\n'))
            if file in ['positive', 'negative']:
                tag.append(file)
                d = {'text': text, 'tag': tag}
            else:
                d = {'text': text}
        dfs.append(pd.DataFrame(data = d))
    return [pd.concat([dfs[0], dfs[1]], axis=0), dfs[2]]

def process(p, tokenizer, text):
    """ Applies standard pre-processing to given text.
    
    Args:
        p (gensim.parsing.porter.PorterStemmer): stemmer object.
        tokenizer (nltk.tokenize.regexp.RegexpTokenizer): tokenizr object.
        text (str): text to preprocess.
    
    Returns:
        list: preprocessed text.
    
    """
    # Converts to lowercase
    doc_nor = text.lower()
    
    # Removes stopwords
    doc_sw = remove_stopwords(doc_nor)
    
    # Stems text
    doc_stem = p.stem_sentence(doc_sw)
    
    # Lemmatizes text
    # TODO: Lemmatizer
    
    # Returns preprocessed text
    return tokenizer.tokenize(doc_stem)

In [87]:
[df, df_unlabeled] = create_labeled_dataset('books')

In [91]:
""" Creates dictionary object and file from preprocessed text.

Args:
    doc_list (list): list with dictionary per naf file.

Returs:
    gensim.corpora.dictionary.Dictionary: complete dictionary.

"""

# Creates PorterStemmer instance.
p = PorterStemmer()

# Creates RegexpTokenizer (removes punctuation signs)
tokenizer = nltk.RegexpTokenizer(r'\w+')

docDict = []
for doc in df_books['text']:
    docDict.append(process(p, tokenizer, doc))
dictionary = corpora.Dictionary(docDict)
dictionary.save('vocab.dict')
""" Creates corpus efficiently based on doc_list

Args:
    doc_list (list): list with dictionary per naf file.
    
Returns: 
    (gensim.corpora.mmcorpus.MmCorpus): doc_corpus file
    (gensim.corpora.mmcorpus.MmCorpus): query_corpus file

"""

doc_corpus = []
for doc in doc_list:
    doc_corpus.append(dictionary.doc2bow(process(p, tokenizer, doc)))

# Serializes and saves dictionary file
corpora.MmCorpus.serialize("doc_corpus.mm",doc_corpus)

In [92]:
print(dictionary)

Dictionary(19794 unique tokens: ['accid', 'book', 'bridget', 'brillant', 'chick']...)
